In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")

data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill")
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [2]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words

35179

In [3]:
tags = list(set(data["Tag"].values))
n_tags = len(tags); n_tags

17

In [4]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [5]:
getter = SentenceGetter(data)

In [6]:
sent = getter.get_next()

In [7]:
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [8]:
# get(word, POS, tag) typle_seq for each sent

In [9]:
sentences = getter.sentences

In [10]:
max_len = 75

In [11]:
#Create tag- tab_ID dictionary

In [12]:
tag2idx = {t:i for i, t in enumerate(tags)}

In [13]:
#Create word seq dy sentence & pad each sent to max_length

In [14]:
sent_word_seq = [[tuple[0] for tuple in s] for s in sentences]

In [15]:
sent_word_seq_new = []

for seq in sent_word_seq:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("__PAD__")
    
    sent_word_seq_new.append(new_seq) 
    
sent_word_seq = sent_word_seq_new

In [16]:
#turn labels into label-IDs & pad with 'O'
label_id_seq = [[tag2idx[w[2]] for w in s] for s in sentences]

In [17]:
#from keras.preprocessing.sequence import pad_sequences
#label_id_seq = pad_sequences(maxlen=max_len, sequences=label_id_seq, padding="post", value=tag2idx["O"])
label_id_seq_new = []

for seq in label_id_seq:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("O")
            
    label_id_seq_new.append(new_seq)

label_id_seq = label_id_seq_new

In [18]:
#Training

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_tr, X_te, y_tr, y_te = train_test_split(sent_word_seq, label_id_seq, test_size=0.2, random_state=2018)

In [21]:
#Applying ELMo Model

In [22]:
batch_size = 32

In [ ]:
import tensorflow as tf

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

SystemError: <class '_frozen_importlib._ModuleLockManager'> returned a result with an error set

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [29]:
import tensorflow_hub as hub

In [30]:
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

AttributeError: module 'tensorflow' has no attribute 'init_scope'

In [ ]:
from keras import backend as K
sess = tf.Session()
K.set_session(sess)

In [31]:
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

RuntimeError: variable_scope module_4/ was unused but the corresponding name_scope was already taken.

In [32]:
def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [33]:
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda

In [35]:
input_text = Input(shape=(max_len,), dtype=tf.string)

embedding = Lambda(ElmoEmbedding, output_shape=(None, max_len, 1024))(input_text)

x = Bidirectional(LSTM(units=512, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)

x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)

x = add([x, x_rnn])  # residual connection to the first biLSTM

out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)

NameError: name 'elmo_model' is not defined

In [36]:
model = Model(input_text, out)

NameError: name 'out' is not defined

In [37]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

NameError: name 'model' is not defined

In [38]:
X_tr, X_val = X_tr[:1213*batch_size], X_tr[-135*batch_size:]
y_tr, y_val = y_tr[:1213*batch_size], y_tr[-135*batch_size:]
y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)

In [39]:
history = model.fit(np.array(X_tr), y_tr, validation_data=(np.array(X_val), y_val),
                    batch_size=batch_size, epochs=5, verbose=1)

NameError: name 'model' is not defined

In [40]:
hist = pd.DataFrame(history.history)

NameError: name 'history' is not defined

In [41]:
plt.figure(figsize=(12,12))
plt.plot(hist["acc"])
plt.plot(hist["val_acc"])
plt.title("Learning curves")
plt.legend()
plt.show()

NameError: name 'hist' is not defined

<Figure size 864x864 with 0 Axes>

In [42]:
i = 19
p = model.predict(np.array(X_te[i:i+batch_size]))[0]
p = np.argmax(p, axis=-1)
print("{:15} {:5}: ({})".format("Word", "Pred", "True"))
print("="*30)
for w, true, pred in zip(X_te[i], y_te[i], p):
    if w != "__PAD__":
        print("{:15}:{:5} ({})".format(w, tags[pred], tags[true]))

NameError: name 'model' is not defined